## Chapter 22: gRPC vs. SignalR vs. Web APIs

In modern distributed systems, choosing the right communication protocol is critical. Your decision affects performance, scalability, developer productivity, and the types of clients you can support. ASP.NET Core supports three primary communication technologies: **Web APIs** (REST/HTTP), **SignalR** (real‑time), and **gRPC** (high‑performance RPC). Each has its strengths and ideal use cases. In this chapter, you’ll learn the characteristics of each, compare them, and understand when to use which. You’ll also build a simple gRPC service to see it in action. By the end, you’ll be able to make informed architectural decisions for your applications.

### 22.1 Web APIs (REST over HTTP)

Web APIs are the most familiar and widely used approach. They follow the REST architectural style (or a pragmatic subset) and communicate over HTTP/1.1 or HTTP/2, typically using JSON as the data format.

#### Characteristics

- **Request‑response model**: Clients send HTTP requests (GET, POST, PUT, DELETE) and servers respond.
- **Stateless**: Each request is independent; the server does not maintain client state.
- **Text‑based (JSON/XML)**: Human‑readable, easy to debug, and widely supported.
- **Caching**: HTTP caching headers can be used to improve performance.
- **Broad client support**: Any client that can make HTTP requests (browsers, mobile apps, IoT devices) can consume a Web API.
- **Tools and ecosystem**: Rich tooling (Swagger, Postman), easy to document, and a mature ecosystem.

#### When to Use Web APIs

- Public APIs intended for external developers.
- When you need broad client compatibility.
- CRUD operations and resource‑oriented services.
- When simplicity and ease of use outweigh raw performance.

### 22.2 SignalR (Real‑Time Communication)

SignalR is a library that simplifies adding real‑time web functionality. It abstracts the underlying transport (WebSockets, Server‑Sent Events, Long Polling) and provides a simple API for server‑to‑client and client‑to‑server communication.

#### Characteristics

- **Full‑duplex, persistent connections**: The server can push data to clients at any time.
- **Transport abstraction**: Automatically selects the best transport (WebSockets preferred).
- **Hub pattern**: Clients call methods on the server, and the server can call methods on clients.
- **Scalable**: Can use a backplane (Redis, Azure SignalR Service) for multi‑server deployments.
- **Protocols**: JSON and MessagePack (binary) for efficient serialization.

#### When to Use SignalR

- Live dashboards (e.g., real‑time sales data).
- Chat applications.
- Collaborative editing (like Google Docs).
- Notifications and alerts.
- Gaming or any scenario requiring low‑latency updates from server to client.

### 22.3 gRPC (High‑Performance RPC)

gRPC is a modern, high‑performance RPC (Remote Procedure Call) framework developed by Google. It uses HTTP/2 for transport, Protocol Buffers (Protobuf) as the interface definition language and message serialization, and supports bi‑directional streaming.

#### Characteristics

- **HTTP/2**: Multiplexed, binary, efficient.
- **Protocol Buffers**: Strongly typed, compact binary serialization (faster and smaller than JSON).
- **Code generation**: You define services and messages in `.proto` files, then generate server and client code in multiple languages.
- **Streaming**: Supports four kinds of methods: unary (request‑response), server streaming, client streaming, and bidirectional streaming.
- **Strongly typed**: Compile‑time safety.
- **Performance**: Very low latency and high throughput, ideal for internal microservices.

#### When to Use gRPC

- Internal microservices communication (where both ends are under your control).
- Polyglot environments (different languages need to communicate efficiently).
- Low‑latency, high‑throughput scenarios.
- Real‑time streaming needs (but SignalR is often easier for browser clients).
- When you need strong API contracts and code generation.

**Note:** gRPC is not directly supported by browsers because browsers cannot enforce HTTP/2 requirements for gRPC (though there is gRPC‑Web as an alternative). For browser clients, you might use gRPC‑Web or combine gRPC with a Web API gateway.

### 22.4 Comparison Table

| Feature                | Web API (REST/HTTP)          | SignalR                      | gRPC                         |
|------------------------|-------------------------------|------------------------------|------------------------------|
| **Communication Model**| Request‑response              | Real‑time, bi‑directional    | RPC (unary, streaming)       |
| **Transport**          | HTTP/1.1, HTTP/2              | WebSockets, SSE, Long Polling| HTTP/2                        |
| **Data Format**        | JSON, XML, etc. (text)        | JSON, MessagePack            | Protocol Buffers (binary)     |
| **Client Support**     | Universal (browsers, mobile, etc.) | Browsers, .NET, others via client libs | .NET, Java, Python, Go, etc. (not direct browser) |
| **Performance**        | Good (but text overhead)      | Good (persistent connections) | Excellent (binary, multiplexed) |
| **Use Case**           | Public APIs, CRUD             | Real‑time updates, chat      | Internal microservices, streaming |
| **Learning Curve**     | Low                           | Medium                       | Medium (Protobuf, code gen)   |
| **Tooling**            | Swagger, Postman              | Built‑in, browser tools      | `protoc`, `grpcurl`           |

### 22.5 Creating a Simple gRPC Service in ASP.NET Core

Let’s build a simple gRPC service to understand the workflow.

#### Step 1: Create a gRPC Project

Use the gRPC template:

```bash
dotnet new grpc -n MyGrpcService
```

This creates a project with a sample `Greeter` service.

#### Step 2: Define the Proto File

The `Protos` folder contains `.proto` files. Let’s define a simple product service.

**Protos/products.proto**:

```protobuf
syntax = "proto3";

option csharp_namespace = "MyGrpcService";

package product;

service ProductService {
  rpc GetProduct (ProductRequest) returns (ProductReply);
  rpc ListProducts (Empty) returns (stream ProductReply);
}

message ProductRequest {
  int32 id = 1;
}

message ProductReply {
  int32 id = 1;
  string name = 2;
  double price = 3;
}

message Empty {}
```

#### Step 3: Generate Code

The project file already includes the `Protobuf` item to generate code:

```xml
<ItemGroup>
  <Protobuf Include="Protos\products.proto" GrpcServices="Server" />
</ItemGroup>
```

When you build the project, the C# code for the service base classes and message types is generated.

#### Step 4: Implement the Service

Create a new class `ProductService` that inherits from the generated base class.

```csharp
using Grpc.Core;
using MyGrpcService;

public class ProductService : ProductServiceBase
{
    private readonly ILogger<ProductService> _logger;
    private static readonly List<ProductReply> _products = new()
    {
        new ProductReply { Id = 1, Name = "Laptop", Price = 999.99 },
        new ProductReply { Id = 2, Name = "Mouse", Price = 19.99 }
    };

    public ProductService(ILogger<ProductService> logger)
    {
        _logger = logger;
    }

    public override Task<ProductReply> GetProduct(ProductRequest request, ServerCallContext context)
    {
        var product = _products.FirstOrDefault(p => p.Id == request.Id);
        if (product == null)
        {
            throw new RpcException(new Status(StatusCode.NotFound, "Product not found"));
        }
        return Task.FromResult(product);
    }

    public override async Task ListProducts(Empty request, IServerStreamWriter<ProductReply> responseStream, ServerCallContext context)
    {
        foreach (var product in _products)
        {
            await responseStream.WriteAsync(product);
            await Task.Delay(1000); // simulate streaming
        }
    }
}
```

#### Step 5: Register the Service

In `Program.cs`, map the gRPC service:

```csharp
app.MapGrpcService<ProductService>();
app.MapGet("/", () => "Communication with gRPC endpoints must be made through a gRPC client.");
```

#### Step 6: Create a gRPC Client (Optional)

Create a console client project and add the same proto file with `GrpcServices="Client"`. Then write a client to call the service.

```csharp
using var channel = GrpcChannel.ForAddress("https://localhost:5001");
var client = new ProductService.ProductServiceClient(channel);

// Unary call
var reply = await client.GetProductAsync(new ProductRequest { Id = 1 });
Console.WriteLine($"Product: {reply.Name} - {reply.Price}");

// Streaming call
var cts = new CancellationTokenSource();
var streamingCall = client.ListProducts(new Empty(), cancellationToken: cts.Token);
await foreach (var product in streamingCall.ResponseStream.ReadAllAsync())
{
    Console.WriteLine($"Streamed: {product.Name}");
}
```

### 22.6 gRPC vs. SignalR vs. Web API – Decision Guide

- **Use Web API** when:
  - You need a public API for external clients.
  - Simplicity and wide compatibility are paramount.
  - You want to leverage HTTP caching, RESTful design.
  - Clients are browsers, mobile apps, or any HTTP client.

- **Use SignalR** when:
  - You need real‑time updates from server to client (or client to server).
  - You’re building features like live chat, dashboards, notifications.
  - Your clients are browsers or .NET applications (SignalR client libraries exist for many platforms).

- **Use gRPC** when:
  - You’re building internal microservices that need high performance.
  - You have a polyglot environment and want strong contracts with code generation.
  - You need streaming (server, client, bidirectional).
  - Low latency and efficient binary serialization are critical.

#### Hybrid Approaches

- **gRPC for internal, Web API for external**: Many organizations expose a public REST API while using gRPC for service‑to‑service communication behind the scenes.
- **SignalR for real‑time, Web API for CRUD**: Keep your real‑time layer separate from standard data operations.

### 22.7 Performance Considerations

- **Web APIs**: JSON serialization/deserialization adds overhead. For high‑volume internal traffic, this can become a bottleneck.
- **SignalR**: Persistent connections consume server resources. Use with a backplane for scale.
- **gRPC**: Extremely efficient, but requires HTTP/2 and may not be suitable for browser clients without a proxy.

### 22.8 Summary

In this chapter, you’ve explored three core communication technologies in ASP.NET Core:

- **Web APIs** – The universal choice for public APIs and RESTful services.
- **SignalR** – The go‑to for real‑time, bi‑directional communication with browsers and clients.
- **gRPC** – The high‑performance RPC framework for internal microservices and streaming.

Each has its place, and understanding their strengths helps you architect systems that are both performant and maintainable. Often, they are used together in a single application to serve different purposes.

**Exercise:**

1. Create a new gRPC service with a method that returns a list of orders. Implement both unary and server‑streaming versions.
2. Write a .NET client to consume the service and display the orders.
3. Compare the performance of a simple JSON Web API vs. gRPC for a small data set (use `Stopwatch`). Note the differences.
4. If you have a SignalR application, consider adding a gRPC endpoint for internal monitoring or data processing.

In the next chapter, **"Containerization with Docker,"** you’ll learn how to package your ASP.NET Core applications into containers for consistent deployment across environments. We’ll cover creating Dockerfiles, using docker‑compose for multi‑container applications, and best practices for containerizing .NET apps.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. aspnet_core_with_a_modern_javascript_frontend.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='23. containerization_with_docker.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
